In [66]:
import os
 #-----------------------------  open import ---------------------------------------#------ 
import cv2
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from skimage.io import imread,imshow
from skimage import measure,color,io,feature
from scipy.ndimage.morphology import binary_fill_holes
import matplotlib.patches as mpatches
from skimage.morphology import convex_hull_image
from skimage.measure import label, regionprops
import math
from scipy import ndimage as ndi
from skimage import feature
from skimage.filters import threshold_mean
from copy import copy,deepcopy
%matplotlib inline

In [68]:
x=[]
y_true=[]
path='C:/Users/Bermuda/OneDrive/Workshop/Dataset'
for fname in os.listdir(path):
    print(fname,end=' ')
    paths=path+'/'+str(fname)
    for iname in os.listdir(paths):
        #print(iname,end=' ')
        pathss=paths+'/'+str(iname)
        img=imread(pathss)
        img=cv2.resize(img,(100,100))
        img_hsv=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)


        #convert rgb to hsv
        img_hsv=cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
        
        #select green only
        img_green=deepcopy(img_hsv)
        img_green[:,:,0]=0
        img_green[:,:,2]=0

        #convert hsv green only to grayscale
        img_gray0=img_green[:,:,1]

        #thresholding
        ret,thresh = cv2.threshold(img_gray0,50,255,cv2.THRESH_BINARY)

        #morprology (dilation)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3)) 
        dilation = cv2.dilate(thresh,kernel,iterations = 1)
        dilation[99:,:]=1
        dilation[30:60,99:]=1

        #fill holes
        filled_I = binary_fill_holes(dilation)
        filled_I[99:,:]=0  
        filled_I[30:60,99:]=0

        #------------------------------------------------------------
        #feather
        feather_anale=[]
        feather_numberOfFingerTip=[]
        feather_defect=[]
        feather_hull=[]
        feather_ratio_area_hull=[]
        #------------------------------------------------------------
        filled_I = filled_I*1

        #find contour
        im2,contours,h= cv2.findContours(filled_I,2,1)
        cnt = contours[0]

        #find contour area
        contourArea=cv2.contourArea(cnt)

        #find convex hull
        hull_point = cv2.convexHull(cnt,returnPoints = True)
        hull_length = cv2.convexHull(cnt,returnPoints = False)
        feather_hull.append(hull_length)
        hullArea=cv2.contourArea(hull_point)
        if(hullArea==0):
            ratio = 0
        else:
            ratio=contourArea/hullArea
        
        feather_ratio_area_hull.append(ratio)

        defects = cv2.convexityDefects(cnt,hull_length)
        try:
            defects.shape
            print("checked for shape".format(defects.shape))
        
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])
                cv2.line(img,start,end,[153,204,255],2) #0,255,0
                cv2.circle(img,far,1,[0,0,255],-1)
            #********************************************************************

            #find center
            M = cv2.moments(cnt)
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            center = (cx,cy)
            cv2.circle(img,center,1,[255,0,0],-1)
            # cv2.Line(img,(cx,cy),1,[255,0,0],-1)
            count_defects = 0
            lena = []
            lenfinger = []
            maxi = 0
            check = 0
            prelen=0
            #find defect
            print(defects.shape[0])
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])

                # find length of all sides of triangle
                #start to end
                a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                #start to defect
                b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                #defect to end
                c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

                # apply cosine rule here
                angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57

                #center to start check 1 
                longlen = math.sqrt((start[0] - cx)**2 + (start[1] - cy)**2)
                if longlen > maxi:
                    maxi = longlen
                    tip = start

                # ignore angles > 90 and highlight rest with red dots
                if angle <= 92:
                    count_defects += 1
                    cv2.circle(img, far, 1, [255,0,127], -1)
                    #dist = cv2.pointPolygonTest(cnt,far,True)

                    # draw a line from start to end i.e. the convex points (finger tips)
                    # (can skip this part)

                    cv2.line(img,start, end, [255,153,153], 2)
                    cv2.circle(img, start, 1, [153,51,255], -1)
                    cv2.circle(img, end, 1, [153,51,255], -1)
                    lena.append(a)
                    longlen1 = math.sqrt((start[0] - cx)**2 + (start[1] - cy)**2)
                    longlen2 = math.sqrt((end[0] - cx)**2 + (end[1] - cy)**2)
                    if count_defects == 1:
                        lenfinger.append(longlen1)
                        lenfinger.append(longlen2)
                        cv2.line(img,center,start,[255,0,0],1)
                        cv2.line(img,center,end,[255,0,0],1)
                    else:
                        lenfinger.append(longlen2)
                        cv2.line(img,center,end,[255,0,0],1)
            if len(tip)<2:
                tippp = 0
            else:
                tippp=tip[1]
            if len(lena)<1:
                lenaa0 = 0
            else:
                lenaa0=lena[0]
            if len(lena)<2:
                lenaa1 = 0
            else:
                lenaa1=lena[1]
            listJaaa=[count_defects,cy,tippp,lenaa0,lenaa1]
            x.append(listJaaa)
            y_true.append(fname)
        except AttributeError:
            print("shape not found")

0 checked for shape
8
checked for shape
9
checked for shape
9
checked for shape
7
checked for shape
8
checked for shape
13
checked for shape
1
checked for shape
3
shape not found
checked for shape
9
checked for shape
9
checked for shape
13
checked for shape
13
checked for shape
12
checked for shape
9
checked for shape
9
checked for shape
16
checked for shape
10
checked for shape
13
checked for shape
9
checked for shape
11
checked for shape
9
checked for shape
11
checked for shape
12
checked for shape
7
checked for shape
10
checked for shape
11
checked for shape
9
checked for shape
9
checked for shape
18
checked for shape
8
checked for shape
7
checked for shape
10
checked for shape
7
checked for shape
10
checked for shape
14
checked for shape
12
checked for shape
9
checked for shape
8
checked for shape
1
checked for shape
13
1 checked for shape
7
checked for shape
6
checked for shape
7
checked for shape
14
checked for shape
10
checked for shape
10
checked for shape
14
checked for shape


8 checked for shape
8
checked for shape
8
checked for shape
8
checked for shape
8
checked for shape
1
checked for shape
1
checked for shape
8
checked for shape
8
checked for shape
9
checked for shape
9
checked for shape
8
checked for shape
8
shape not found
shape not found
shape not found
shape not found
checked for shape
7
checked for shape
7
checked for shape
9
checked for shape
9
shape not found
shape not found
checked for shape
10
checked for shape
10
checked for shape
1
checked for shape
1
checked for shape
1
checked for shape
1
checked for shape
1
checked for shape
1
shape not found
shape not found
checked for shape
2
checked for shape
2
checked for shape
1
checked for shape
1
shape not found
shape not found
checked for shape
1
checked for shape
1
checked for shape
1
checked for shape
1
checked for shape
3
checked for shape
3
checked for shape
3
checked for shape
3
checked for shape
1
checked for shape
1
checked for shape
8
checked for shape
8
checked for shape
9
checked for shap

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler, minmax_scale
from sklearn.neural_network import MLPClassifier


from sklearn.model_selection import LeaveOneOut
x_norm = minmax_scale(x)


In [78]:
#SVC

sum=0
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x_norm, y_true,test_size=0.3, random_state=i)
    svc = svm.SVC(kernel='linear', C=10).fit(x_train, y_train)
    
    #np.array can call array fill index array background C 
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)

    loo = LeaveOneOut()
    loo.split(x_train)
    
    for train, test in loo.split(x_train):
        svc.fit(x_train[train], y_train[train])
        y_pred = svc.predict(x_train[test])
        
    y_pred=svc.predict(x_test)
    print("Accuracysvc:",metrics.accuracy_score(y_test, y_pred))
    sum += metrics.accuracy_score(y_test, y_pred)
    # print(confusion_matrix(y_test, y_pred))  
    # print(classification_report(y_test, y_pred))
y_pred = svc.predict(x_test)
print("Accuracysvc:",metrics.accuracy_score(y_test, y_pred))    
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracysvc: 0.46564885496183206
Accuracysvc: 0.46564885496183206
Accuracysvc: 0.4198473282442748
Accuracysvc: 0.46564885496183206
Accuracysvc: 0.4580152671755725
Accuracysvc: 0.5038167938931297
Accuracysvc: 0.4351145038167939
Accuracysvc: 0.46564885496183206
Accuracysvc: 0.48091603053435117
Accuracysvc: 0.48854961832061067
Accuracysvc: 0.48854961832061067
[[13  1  0  0  0  0  0  0  0  0]
 [ 0 11  3  1  0  0  0  0  1  0]
 [ 1  0  9  0  0  0  3  1  0  0]
 [ 1  0  0  6  0  0  0  0  1  0]
 [ 2  0  2  0  0  0  7  0  0  3]
 [ 4  0  0  0  0  4  2  0  0  1]
 [ 0  0  0  1  0  0 11  0  0  0]
 [ 0  0  0  1  0  0  0  3  2  3]
 [ 1  0  0  2  0  1 10  2  0  3]
 [ 1  0  3  1  0  1  0  0  1  7]]
             precision    recall  f1-score   support

          0       0.57      0.93      0.70        14
          1       0.92      0.69      0.79        16
          2       0.53      0.64      0.58        14
          3       0.50      0.75      0.60         8
          4       0.00      0.00      0.00  

C:\Users\Bermuda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
#MLP neural network


sum=0
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x_norm, y_true,test_size=0.3, random_state=i)
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                             hidden_layer_sizes=(10, 14), random_state=i)
    #np.array can call array fill index array background C 
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)

    loo = LeaveOneOut()
    loo.split(x_train)
    for train, test in loo.split(x_train):
        clf.fit(x_train[train], y_train[train])
        y_pred = clf.predict(x_train[test])
        
    y_pred=clf.predict(x_test)
    print("Accuracysvc:",metrics.accuracy_score(y_test, y_pred))
    sum += metrics.accuracy_score(y_test, y_pred)
    # print(confusion_matrix(y_test, y_pred))  
    # print(classification_report(y_test, y_pred))
y_pred = clf.predict(x_test)
print("Accuracysvc:",metrics.accuracy_score(y_test, y_pred))    
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracysvc: 0.44274809160305345
Accuracysvc: 0.5114503816793893
Accuracysvc: 0.46564885496183206


In [ ]:
from sklearn.neighbors import KNeighborsClassifier 


sum=0
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x_norm, y_true,test_size=0.30, random_state=i)
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    
    loo = LeaveOneOut()
    loo.split(x_train)
    for train, test in loo.split(x_train):
        knn = KNeighborsClassifier(n_neighbors=10)  
        knn.fit(x_train[train], y_train[train]) 
        y_pred = knn.predict(x_train[test])  
    y_pred=knn.predict(x_test)
    print("Accuracysvc:",metrics.accuracy_score(y_test, y_pred))
    sum += metrics.accuracy_score(y_test, y_pred)
    # print(confusion_matrix(y_test, y_pred))  
    # print(classification_report(y_test, y_pred))
y_pred = knn.predict(x_test)
print("Accuracysvc:",metrics.accuracy_score(y_test, y_pred))    
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))